# Introduction

Ce TP continue le TP précédent. Nous allons reprendre d'ailleurs les mêmes données et commencer la mise en oeuvre d'un modèle de Markov pour la prédiction des étiquettes: 
* une observation est une phrase, représentée comme une séquence de variables aléatoires, une par mot de la phrase
* à cette observation est associée une séquence de variables aléatoires représentant les étiquettes, une par mot de la phrase également

On suppose que la séquence d'observation (une phrase) est générée par un modèle de Markov caché. Les variables cachées sont donc les étiquettes à inférer. Nous allons commencer par écrire une classe python pour représenter le HMM. Cette classe évoluera au fil des TPs. 

Pour cela le code de départ suivant est donné: 

In [26]:
import nltk
from numpy import array, ones, zeros
import sys

# Some words in test could be unseen during training, or out of the vocabulary (OOV) even during the training. 
# To manage OOVs, all words out the vocabulary are mapped on a special token. 
UNK = "<unk>" 
UNKid = 0 

class HMM:
        def __init__(self, state_list, observation_list,
                 transition_proba = None,
                 observation_proba = None,
                 initial_state_proba = None, smoothing_obs=0.01):
            """Builds a new Hidden Markov Model
            state_list is the list of state symbols [q_0...q_(N-1)]
            observation_list is the list of observation symbols [v_0...v_(M-1)]
            transition_proba is the transition probability matrix
                [a_ij] a_ij = Pr(Y_(t+1)=q_i|Y_t=q_j)
            observation_proba is the observation probablility matrix
                [b_ki] b_ki = Pr(X_t=v_k|Y_t=q_i)
            initial_state_proba is the initial state distribution
                [pi_i] pi_i = Pr(Y_0=q_i)"""
            print "HMM creating with: "
            self.N = len(state_list) # The number of states
            self.M = len(observation_list) # The number of words in the vocabulary
            print str(self.N)+" states"
            print str(self.M)+" observations"
            self.omega_Y = state_list # Keep the vocabulary of tags
            self.omega_X = observation_list # Keep the vocabulary of tags
            # Init. of the 3 distributions : observation, transition and initial states
            if transition_proba is None:
                self.transition_proba = zeros( (self.N, self.N), float) 
            else:
                self.transition_proba=transition_proba
            if observation_proba is None:
                self.observation_proba = zeros( (self.M, self.N), float) 
            else:
                self.observation_proba=observation_proba
            if initial_state_proba is None:
                self.initial_state_proba = zeros( (self.N,), float ) 
            else:
                self.initial_state_proba=initial_state_proba
            # Since everything will be stored in numpy arrays, it is more convenient and compact to 
            # handle words and tags as indices (integer) for a direct access. However, we also need 
            # to keep the mapping between strings (word or tag) and indices. 
            self.make_indexes()
            self.smoothing_obs = smoothing_obs

        def make_indexes(self):
            """Creates the reverse table that maps states/observations names
            to their index in the probabilities arrays"""
            self.Y_index = {}
            for i in range(self.N):
                self.Y_index[self.omega_Y[i]] = i
            self.X_index = {}
            for i in range(self.M):
                self.X_index[self.omega_X[i]] = i
                
        def observation_estimation(self, pair_counts):
            """
            Build the observation distribution:
            observation_proba is the observation probability matrix
            [b_ki], b_ki = Pr(X_t = v_k | Y_t = q_i)
            """
            #fill with counts
            for pair in pair_counts:
                wrd = pair[0]
                tag = pair[1]
                cpt = pair_counts[pair]
                k=0
                if wrd in self.X_index:
                    k = self.X_index[wrd]
                i = self.Y_index[tag]
                self.observation_proba[k,i] = cpt
            #normalize
            self.observation_proba = self.observation_proba + self.smoothing_obs
            self.observation_proba = self.observation_proba/self.observation_proba.sum(axis = 0).reshape(1,self.N)
        
        def transition_estimation(self, trans_counts):
            """
            Build the transition distribution:
            transition_proba is the transition probablity matrix
            [a_ij], a[i,j] = Pr(Y_(t+1) = q_i| Y_t = q_j)
            """
            #fill with counts
            for pair in trans_counts:
                i = self.Y_index[pair[1]]
                j = self.Y_index[pair[0]]
                self.transition_proba[i,j] = trans_counts[pair]
            #normalize
            self.transition_proba = self.transition_proba/self.transition_proba.sum(axis=0).reshape(1, self.N)
        
        def init_estimation(self, init_counts):
            """
            Build the init distribution
            """
            for tag in init_counts:
                i = self.Y_index[tag]
                self.initial_state_proba[i]=init_counts[tag]
        
            self.initial_state_proba = self.initial_state_proba/sum(self.initial_state_proba)
                
        def supervised_training(self, pair_counts, trans_counts, init_counts):
            """train the HMM's params"""
            self.observation_estimation(pair_counts)
            self.transition_estimation(trans_counts)
            self.init_estimation(init_counts)
            




# Exercice : Algorithme de Viterbi

La question qui se pose est comment calculer la meilleure séquence d'étiquettes pour une phrase donnée connaissant les paramètres du HMM. Par meilleure, on entend la séquence d'étiquettes (ou d'états) la plus probable connaissant la séquence d'obervation. 

Proposer et implémenter un algorithme répondant à cette question. Pour vous aider à démarrer, cet algorithme s'appelle Viterbi et regardez cette vidéo https://www.youtube.com/watch?v=RwwfUICZLsA, pour comprendre comment il opère. 

# TODO pour le 18/10/2016

* Finir la partie interface (qui comprend l'apprentissage supervisé)
* Regarder la vidéo et implémenter l'algorithme de Viterbi

# TODO pour le 08/11/2016

* Calculer le taux d'erreur de prédiction sur les données de test
    * Implémenter une fonction effectuant l'inférence sur un jeu de donnée et qui calcul le taux d'erreur
* Il peut être avantageux de lisser les distributions d'observation. Expliquer pourquoi et implémenter le 
* Choisir la bonne valeur de lissage sur les données de développement (en regardant le taux d'erreur) et constater l'effet sur les données de test



# Interface avec les données et apprentissage supervisé

Ainsi pour initialiser un HMM, nous allons devoir lire les données (chose faîte lors du TP précédent): 
* écrire une fonction permettant d'initialiser le HMM à partir des données d'apprentissage
* écrire une fonction *apprentissage_supervisé* qui permet d'estimer les paramètres 

Dans un premier temps, nous limiterons, comme lors du TP précédent, le vocabulaire aux mots apparaissant 10 fois ou plus. Les autres mots sont tous remplacés par la même forme *unk*

Pour cela, le plan de travail peut être envisagé ainsi: 
* Lire les données puis générer un corpus de **train** (80%) puis de **test** (10%)
* écrire une fonction qui créer à partir des données d'apprentissage (**train**), tous les comptes nécessaires pour l'estimation supervisée des paramètres du HMM
* écrire 3 fonctions qui estimes les paramètres à partir des comptes, une fonction par distribution: observation, transition, état initial. 
* écrire une fonction qui reprend le tout et qui estime tous les paramètres du HMM


In [13]:
#compter les mots et les tags
def make_count(corpus):
    """
    Build different count tables to train a HMM. Each conut table is a dictionary.
    Returns:
    * c_words: word counts
    * c_tags: tag counts
    * c_pairs: count of pairs (word, tag)
    * c_transitions: count of tag bigram
    * c_inits: count of tag found in the first position
    """
    c_words = dict()
    c_tags = dict()
    c_pairs = dict()
    c_transitions = dict()
    c_inits = dict()
    for sent in corpus:
        #we use i because of the transition counts
        for i in range(len(sent)):
            couple = sent[i]
            wrd = couple[0]
            tag = couple[1]
            #word counts
            if wrd in c_words:
                c_words[wrd] = c_words[wrd] + 1
            else:
                c_words[wrd] = 1
            #tag counts
            if tag in c_tags:
                c_tags[tag] = c_tags[tag] + 1
            else:
                c_tags[tag] = 1
            #observation counts
            if couple in c_pairs:
                c_pairs[couple] = c_pairs[couple] + 1
            else:
                c_pairs[couple] = 1
            #i>0 -> transition counts
            if i>0:
                trans = (sent[i-1][1], tag)
                if trans in c_transitions:
                    c_transitions[trans] = c_transitions[trans] + 1
                else:
                    c_transitions[trans] = 1
            # i ==0 -> counts for initial states
            else:
                if tag in c_inits:
                    c_inits[tag] = c_inits[tag] + 1
                else:
                    c_inits[tag] = 1
    
    return c_words, c_tags, c_pairs, c_transitions, c_inits
            
            

In [5]:
#creation of vocabulaire
def make_vocab(c_words, threshold):
    vocab = list()
    vocab.append('unk')
    for word in c_words:
        if(c_words[word] >= threshold):
            vocab.append(word)
    return vocab

In [6]:
import nltk
data = nltk.corpus.brown.tagged_sents(tagset='universal')

In [7]:
len(data)

57340

In [8]:
trainpart = 0.8
testpart = 0.1
i = int(len(data)*trainpart)
train = data[0:i]
test = data[i:i+int(len(data)*testpart)]

In [9]:
print "Nombre de phrase de train = "+str(len(train))
print "Nombre de phrase de test ="+str(len(test))

Nombre de phrase de train = 45872
Nombre de phrase de test =5734


In [15]:
cwords, ctags, cpairs, ctrans, cinits = make_count(train)
print "Nombre de mots :"+str(len(cwords))
print "Nombre de tags :"+str(len(ctags))
print "Nombre de pairs :"+str(len(cpairs))
print "Nombre de trans :"+str(len(ctrans))
print "Nombre de init:"+str(len(cinits))


Nombre de mots :51461
Nombre de tags :12
Nombre de pairs :54944
Nombre de trans :143
Nombre de init:12


In [16]:
print ctags
vocab = make_vocab(cwords,10)
print "vocabulaire :"+str(len(vocab))

{u'ADV': 45940, u'NOUN': 241528, u'ADP': 126332, u'PRON': 35550, u'DET': 116989, u'.': 118482, u'PRT': 23316, u'VERB': 150459, u'X': 1205, u'NUM': 13802, u'CONJ': 32177, u'ADJ': 73866}
vocabulaire :7991


#Creation du HMM

In [28]:
hmm = HMM(state_list=ctags.keys(), observation_list=vocab,transition_proba=None,
         observation_proba=None,
         initial_state_proba=None,smoothing_obs=0.001)
hmm.supervised_training(cpairs,ctrans,cinits)

HMM creating with: 
12 states
7991 observations


In [29]:
hmm.transition_proba.sum(axis=0)

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.])

In [34]:
print sum(hmm.initial_state_proba)

1.0
